# DAPI nuclei detection using cellpose

#### This script aims to segment nuclei using DAPI-stained images for downstream analysis. For this purpose, machine learning model 'nuclei' from cellpose is used. It was tested and works pretty well, but:

- the newest version of cellpose (cellpose 4 as of 25.8.2025) is no longer compatible with the nuclei model and only 'cpasm' model downloads (which should be able to handle everything). The problem is, that this model is too computationally heavy for computers without GPU (like the one I use here). At one point I tested it on one image and it took over 20min (after that I interrupted the kernel as there is no point having script that takes that long). That is the reason why the cellpose this script uses is cellpose2.2.3.

- currently takes about ~1min per image

- has problems with very nuclei-dense areas, but is nonetheless very accurate

- the python version is also Python 3.10.18 (older from the base env.), because it's compatible with the version of cellpose this script uses

In [ ]:
### LIBRARY
import os
from skimage import io
import glob
from natsort import natsorted
from skimage.color import label2rgb
from skimage.io import imsave
from statistics import mean, stdev

## cellpose
from cellpose import models

In [ ]:
### PARAMETERS
filename = 'C:/Users/terez/Documents/Skola/Ludwig-Maximilians_Universitat_Munchen/Research course Moretti lab/Microscopy data etc/EXP6/Macrophages_and_ECs/ImmEpis_EXP6_d1_GFP_cTnT_CD31'

# important:
diameter_user_select = 10
# the nuclei model heavily leans on the user-selected diameter (i.e. expected diameter of the nuclei 
# in pixels). Depending on the magnification of the microscope images, this parameter also needs to change.


# filename extract
last_folder = os.path.basename(filename)
# to save masks:
folder1 = f'{filename}/masks_nuclei'
os.makedirs(folder1, exist_ok=True)
# output folder:
output_folder = 'C:/Users/terez/Documents/Skola/Ludwig-Maximilians_Universitat_Munchen/Research course Moretti lab/Microscopy data etc/EXP6/EC_analysis'

print(last_folder)


In [ ]:
### LOADING IMAGES

## DAPI in 405 channel:
list_of_dapi_files = natsorted(glob.glob(f"{filename}/*ch00*.tif"))
images_dapi = {}
images_dapi_list = []
for file in list_of_dapi_files:
    img = io.imread(file)
    images_dapi[os.path.basename(file)] = img
    images_dapi_list.append(img)

print(f"to check: \n number of 405 images: \t {len(images_dapi)}")

In [ ]:
### CELLPOSE implement (nuclei count)

# Load the built-in nuclei model
model = models.Cellpose(model_type="nuclei")

nuclei_masks = {}

quality_check = {}

for name,img in images_dapi.items():
    res = model.eval(img, channels=[0,0], diameter=diameter_user_select) #diameter=None worked well
    mask, flow, style, diam = res

    ## Visualisation
    coloured_mask = label2rgb(mask, bg_label=0, bg_color=(0,0,0))

    ## Saving into dict
    nuclei_masks[name] = {
        'nuclei_mask': mask,
        'colour_mask': coloured_mask
    }

    nuclei_count = mask.max()
    nuclei_sizes = []

    nuclei_objects = regionprops(mask, img)
    for obj in nuclei_objects:
        nucleus_area = obj.area
        nuclei_sizes.append(nucleus_area)

    ## Saving into new folders

    base_name = os.path.splitext(name)[0]

    imsave(os.path.join(folder1, f'{base_name}_mask.tif'), mask.astype(np.uint16))
    imsave(os.path.join(folder1, f'{base_name}_colour_mask.png'), (coloured_mask * 255).astype(np.uint8))

    quality_check[base_name] = {
        'nuclei count': nuclei_count,
        'nuclei size average': mean(nuclei_sizes),
        'nuclei size stdev': stdev(nuclei_sizes)
    }

    

In [ ]:
### DATAFRAME
df_stain1 = pd.DataFrame.from_dict(quality_check, orient='index')

output_path = f'{output_folder}/{last_folder}.xlsx'
df_stain1.to_excel(output_path)